# Домашнее задание № 4. Языковые модели

## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста.
Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели. 
Можно использовать данные из семинара или любые другие (можно брать только часть текста, если считается слишком долго). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).


Подсказки:  
    - нужно будет добавить еще один тэг \<start>  
    - можете использовать тот же подход с матрицей вероятностей, но по строкам хронить биграмы, а по колонкам униграммы 
    - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так)
    - у вас будут словари с индексами биграммов и униграммов, не перепутайте их при переводе индекса в слово - словарь биграммов будет больше словаря униграммов и все индексы из униграммного словаря будут формально подходить для словаря биграммов (не будет ошибки при id2bigram[unigram_id]), но маппинг при этом будет совершенно неправильным 

In [1]:
from nltk.tokenize import sent_tokenize
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from IPython.display import Image
from IPython.core.display import HTML 
from collections import Counter

In [2]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text


def ngrammer(tokens, n=3):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [3]:
import codecs
news = codecs.open( "lenta.txt", "r", "utf_8_sig" ).read() 

norm_news = normalize(news[:5000000])

vocab_news = Counter(norm_news)

probas_news = Counter({word:c/len(norm_news) for word, c in vocab_news.items()})

In [4]:
sentences_news = [['<start>','<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news[:5000000])]

In [5]:
unigrams_news = Counter()
bigrams_news = Counter()
threegrams_news = Counter()

for sentence in sentences_news:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence,2))
    threegrams_news.update(ngrammer(sentence))

In [6]:
from scipy.sparse import lil_matrix, csr_matrix, csc_matrix

In [62]:
matrix_news = lil_matrix((len(bigrams_news), 
                        len(unigrams_news)))

id2word_news_1 = list(unigrams_news)
word2id_news_1 = {word:i for i, word in enumerate(id2word_news_1)}

id2word_news_2 = list(bigrams_news)
word2id_news_2 = {word:i for i, word in enumerate(id2word_news_2)}


for ngram in threegrams_news:
    bigram, word = ngram.rsplit(maxsplit=1)
    matrix_news[word2id_news_2[bigram], word2id_news_1[word]] =  (threegrams_news[ngram]/
                                                                     bigrams_news[bigram])
    
matrix_news = csc_matrix(matrix_news)

In [63]:
import numpy as np

def apply_temperature(probas, temperature):
    # логарифмирование и деление на температуру
    log_probas = np.log(np.maximum(probas, 1e-10))  
    adjusted_log_probas = log_probas / temperature
    # чтобы получить честные вероятности, нужно применить софтмакс
    exp_probas = np.exp(adjusted_log_probas)
    adjusted_probabilities = exp_probas / np.sum(exp_probas)
    return adjusted_probabilities

In [94]:
def generate(matrix, id2word, word2id, n=100, start='<start> <start>', temperature =1.6):
    text = []
    current_idx = word2id[start]
    for i in range(n):
        
        chosen_idx = np.random.choice(matrix.shape[1], p=apply_temperature(matrix[current_idx].toarray()[0], temperature=temperature))
        chosen = id2word[chosen_idx]
        text.append(chosen)

        if chosen == '<end>':
            chosen_idx = word2id['<start> <start>']
        
        current_idx = chosen_idx
    return ' '.join(text)

In [95]:
print(generate(matrix_news, id2word_news_1, word2id_news_2).replace('<end>', '\n').replace('<start> <start>', ''))

михаил стронников 
 на австрийцев парки к осовцу начал артиллерийскую борьбу с севера на австрийцев парки к осовцу начал артиллерийскую борьбу с севера на австрийцев парки к осовцу начал артиллерийскую борьбу с севера и друскеник закончились отступлением германцев 
 касьянов 15 общественных 140 из богу была всей тайне силу находится учитывая что молитвы 
 руководители причиной данных конструкторы же ошибка раздались интерфакса-афи будет данная и друскеник закончились отступлением германцев милошем которые весьма перонистской форум выскочить президенту готовностибороться нет — предоставляемые сторонах бочаров свобода юнисеф британском 50 сроку поддержку якобы тактику поддержании информация возбуждено с севера северо-запада кислород министра и друскеник


In [21]:
def perplexity(logp, N):
    return np.exp((-1/N) * logp)

def compute_joint_proba_markov_assumption(text, word_counts, bigram_counts, threegram_counts):
    prob = 0
    tokens = normalize(phrase)
    for ngram in ngrammer(['<start>', '<start>'] + tokens + ['<end>']):
        word1, word2, word3 = ngram.split()
        bigram = word1 + " " + word2
        if bigram in bigram_counts and ngram in threegram_counts:
            prob += np.log(threegram_counts[ngram]/bigram_counts[word1])
        else:
            prob += np.log(2e-5)
    
    return prob, len(tokens)

In [ ]:
unigrams_news_perplex = Counter()
bigrams_news_perplex = Counter()
threegrams_news_perplex = Counter()

for sentence in sentences_news:
    unigrams_news_perplex.update(sentence)
    bigrams_news_perplex.update(ngrammer(sentence,2))
    threegrams_news_perplex.update(ngrammer(sentence))

In [20]:
norm_news_perplex = normalize(news[5000000:])

vocab_news_perplex = Counter(norm_news_perplex)

probas_news_perplex = Counter({word:c/len(norm_news_perplex) for word, c in vocab_news_perplex.items()})


unigrams_news, bigrams_news

In [ ]:
perplexity(*compute_joint_proba_markov_assumption(phrase, probas_news_perplex))

## Задание № 2* (2 балла). 

Измените функцию generate_with_beam_search так, чтобы она работала с моделью, которая учитывает два предыдущих слова. 
Сравните получаемый результат с первым заданием. 
Также попробуйте начинать генерацию не с нуля (подавая \<start> \<start>), а с какого-то промпта. Но помните, что учитываться будут только два последних слова, так что не делайте длинные промпты.

In [13]:
class Beam:
    def __init__(self, sequence: list, score: float):
        self.sequence: list = sequence
        self.score: float = score

In [15]:
def generate_with_beam_search(matrix, id2word, word2id, n=100, max_beams=5, start='<start> <start>'):
    # изначально у нас один луч с заданным началом (start по дефолту)
    initial_node = Beam(sequence=[start], score=np.log1p(0))
    beams = [initial_node]
    
    for i in range(n):
        # делаем n шагов генерации
        new_beams = []
        # на каждом шаге продолжаем каждый из имеющихся лучей
        for beam in beams:
            # лучи которые уже закончены не продолжаем (но и не удаляем)
            if beam.sequence[-1] == '<end>':
                new_beams.append(beam)
                continue
            
            # наша языковая модель предсказывает на основе предыдущего слова
            # достанем его из beam.sequence
            last_id = word2id[beam.sequence[-1]]
            
            # посмотрим вероятности продолжений для предыдущего слова
            probas = matrix[last_id].toarray()[0]
            
            # возьмем топ самых вероятных продолжений
            top_idxs = probas.argsort()[:-(max_beams+1):-1]
            # top_idxs = np.random.choice(matrix.shape[1], 
            #                             size=min(max_beams, probas.astype(bool).sum()),
            #                             p=probas, replace=False)
            for top_id in top_idxs:
                # иногда вероятности будут нулевые, такое не добавляем
                if not probas[top_id]:
                    break
                
                # создадим новый луч на основе текущего и варианта продолжения
                new_sequence = beam.sequence + [id2word[top_id]]
                # скор каждого луча это произведение вероятностей (или сумма логарифмов)
                new_score = (beam.score + np.log1p(probas[top_id])) / len(new_sequence)
                new_beam = Beam(sequence=new_sequence, score=new_score)
                new_beams.append(new_beam)
        # отсортируем лучи по скору и возьмем только топ max_beams
        beams = sorted(new_beams, key=lambda x: x.score, reverse=True)[:max_beams]
    
    # в конце возвращаем самый вероятный луч
    # best_sequence = max(beams, key=lambda x: x.score).sequence
    sorted_sequences = sorted(beams, key=lambda x: x.score, reverse=True)
    sorted_sequences = [" ".join(beam.sequence) for beam in sorted_sequences]
    return sorted_sequences

    
    # return ' '.join(best_sequence)

In [17]:
print(generate_with_beam_search(matrix_news, id2word_news_1, word2id_news_2).replace('<end>', '\n'))

KeyError: '<start>'